# Theme

This project simulates an autonomous driving AI that reads a driving scenario, decides what action the car should take (accelerate, brake, stop, turn, honk), evaluates risk, and visually shows whether the decision leads to a safe outcome or a crash—learning from past mistakes like a real self-driving brain.

# Install

gradio → Builds an interactive web interface where users enter driving scenarios and see the AI’s decisions.

langchain → Manages structured prompts and connects logic with the language model.

langchain-google-genai → Enables the AI engine to communicate with the Gemini model for decision-making.

In [ ]:
!pip install -q gradio langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.8/475.8 kB 11.2 MB/s eta 0:00:00


Imports Cell → Loads libraries for UI, plotting, AI communication, memory handling, and scenario processing.

In [ ]:
import gradio as gr
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os, json
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage

Gemini API Key set up

Step-1: Go to Google AI Studio https://ai.google.dev/?utm_source=chatgpt.com

Step-2: Sign in with your Google account.

Step-3: Open the “API Keys” page. In the left sidebar, click on “API keys”.

Step-4: Create or select a project.

Step-5: Create a new API key.

Step-6: Copy and store the key safely.

# API Setup

Gemini API Setup Cell → Configures and authenticates the Gemini model so the AI can generate driving decisions.

Memory, Speed, Actions Cell → Initializes the car’s state, available actions, and memory where the AI stores lessons from crashes.

In [ ]:
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('Insert your Gemin Api Key')
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0.2)
except:
    llm = None
    print("❌ API key missing. AI won't respond.")

memory = []
speed = 60
ACTIONS = ["ACCELERATE", "BRAKE", "TURN_LEFT", "TURN_RIGHT", "HONK", "STOP"]

# Visualizer

draw_scene() Function Cell → Visually renders the road, car, AI action, speed, and risk level using Matplotlib.

In [ ]:
def draw_scene(scenario, action, outcome, reasoning, speed, risk):
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_facecolor("#505050")
    ax.set_xlim(0, 12)
    ax.set_ylim(0, 10)
    ax.axis("off")

    ax.add_patch(patches.Rectangle((3, 0), 5, 10, color="#303030"))
    ax.plot([5.5, 5.5], [0, 10], '--', color="yellow")

    car_color = "green" if outcome == "SAFE" else "red"
    ax.add_patch(patches.Rectangle((4.7, 2), 2, 3, facecolor=car_color))
    ax.text(5.7, 3.5, "CAR", fontsize=12, color="white")

    hud = f"Speed: {speed} km/h\nAction: {action}\nRisk: {risk}\nOutcome: {outcome}"
    ax.text(9.5, 7, hud, fontsize=12, bbox=dict(facecolor="white", alpha=0.8))

    ax.text(9.5, 2, "AI Reasoning:\n" + reasoning, fontsize=10,
            bbox=dict(facecolor="white", alpha=0.8))

    return fig

# AI Engine

driving_ai() Function Cell → Sends the driving scenario to Gemini and receives an action with reasoning and a risk score in JSON.

In [ ]:
def driving_ai(scenario):
    global speed, memory

    if not llm:
        return "⚠️ AI Key Missing", None

    past = "\n".join(memory) if memory else "None"

    system = f"""
    You are an autonomous driving brain.
    MEMORY: {past}
    Actions: {ACTIONS}
    Output JSON only:
    {{
      "action": "ACCELERATE|BRAKE|STOP|TURN_LEFT|TURN_RIGHT|HONK",
      "reasoning": "short explanation",
      "risk_score": 0-100
    }}
    """

    try:
        res = llm.invoke([SystemMessage(content=system), HumanMessage(content=scenario)])
        data = json.loads(res.content.replace("```json", "").replace("```", ""))
    except:
        data = {"action": "STOP", "reasoning": "Fallback Safety", "risk_score": 80}

    return data, None

run_simulation() Function Cell → Applies the AI’s chosen action, updates speed, calculates risk, detects crashes, and visualizes the outcome.


In [ ]:
def run_simulation(scenario):
    global speed

    ai, _ = driving_ai(scenario)
    action, reasoning, risk = ai["action"], ai["reasoning"], ai["risk_score"]
    outcome = "SAFE"

    if action == "ACCELERATE": speed += 20
    if action == "BRAKE": speed = max(0, speed - 40)
    if action == "STOP": speed = 0

    if "rain" in scenario.lower(): risk += 30
    if speed > 90: risk += 40

    if risk > 85 and action == "ACCELERATE":
        outcome = "CRASH"
        lesson = f"On '{scenario}', accelerating caused crash."
        if lesson not in memory: memory.append(lesson)

    fig = draw_scene(scenario, action, outcome, reasoning, speed, risk)
    return fig, f"🧠 **Action:** {action}\n📌 **Outcome:** {outcome}\n⚡ **Speed:** {speed} km/h\n💡 **Reasoning:** {reasoning}"

Gradio UI

In [ ]:
with gr.Blocks(title="AI Autonomous Driving Simulator") as ui:
    gr.Markdown("# 🚗 AI Autonomous Driving Agent")
    gr.Markdown("Enter a driving situation and watch the AI make a decision.")

    scenario = gr.Textbox(
        label="Enter Scenario",
        placeholder="e.g., Driving fast on a rainy road"
    )

    output_plot = gr.Plot(label="Driving Visualization")
    output_text = gr.Markdown(label="AI Thinking")

    gr.Button("Run Simulation 🚦").click(
        run_simulation,
        inputs=[scenario],
        outputs=[output_plot, output_text]
    )

ui.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9614228ddde95c9a24.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Driving at 80 km/h during heavy rain.

A child runs across the road suddenly.

On a sunny highway with no cars ahead.

Car in front brakes suddenly.
